In [77]:
import re
import pandas as pd
import numpy as np
from sklearn import metrics
import nltk
from nltk import tokenize, corpus, stem
from gensim.models import word2vec
from IPython.core.interactiveshell import InteractiveShell


In [9]:
# #全部行都能输出
InteractiveShell.ast_node_interactivity = "all"

In [10]:
data = pd.read_csv('/Users/JQC/Desktop/stocknews/Combined_News_DJIA.csv')
data.head(3)
data.tail(3)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."


,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
1986,2016-06-29,1,Explosion At Airport In Istanbul,Yemeni former president: Terrorism is the offs...,UK must accept freedom of movement to access E...,Devastated: scientists too late to captive bre...,British Labor Party leader Jeremy Corbyn loses...,A Muslim Shop in the UK Was Just Firebombed Wh...,Mexican Authorities Sexually Torture Women in ...,UK shares and pound continue to recover,...,"Escape Tunnel, Dug by Hand, Is Found at Holoca...",The land under Beijing is sinking by as much a...,Car bomb and Anti-Islamic attack on Mosque in ...,Emaciated lions in Taiz Zoo are trapped in blo...,Rupert Murdoch describes Brexit as 'wonderful'...,More than 40 killed in Yemen suicide attacks,Google Found Disastrous Symantec and Norton Vu...,Extremist violence on the rise in Germany: Dom...,BBC News: Labour MPs pass Corbyn no-confidence...,Tiny New Zealand town with 'too many jobs' lau...
1987,2016-06-30,1,Jamaica proposes marijuana dispensers for tour...,Stephen Hawking says pollution and 'stupidity'...,Boris Johnson says he will not run for Tory pa...,Six gay men in Ivory Coast were abused and for...,Switzerland denies citizenship to Muslim immig...,Palestinian terrorist stabs israeli teen girl ...,Puerto Rico will default on $1 billion of debt...,Republic of Ireland fans to be awarded medal f...,...,Googles free wifi at Indian railway stations i...,Mounting evidence suggests 'hobbits' were wipe...,The men who carried out Tuesday's terror attac...,Calls to suspend Saudi Arabia from UN Human Ri...,More Than 100 Nobel Laureates Call Out Greenpe...,British pedophile sentenced to 85 years in US ...,"US permitted 1,200 offshore fracks in Gulf of ...",We will be swimming in ridicule - French beach...,UEFA says no minutes of silence for Istanbul v...,Law Enforcement Sources: Gun Used in Paris Ter...
1988,2016-07-01,1,A 117-year-old woman in Mexico City finally re...,IMF chief backs Athens as permanent Olympic host,"The president of France says if Brexit won, so...",British Man Who Must Give Police 24 Hours' Not...,100+ Nobel laureates urge Greenpeace to stop o...,Brazil: Huge spike in number of police killing...,Austria's highest court annuls presidential el...,"Facebook wins privacy case, can track any Belg...",...,"The United States has placed Myanmar, Uzbekist...",S&amp;P revises European Union credit rating t...,India gets $1 billion loan from World Bank for...,U.S. sailors detained by Iran spoke too much u...,Mass fish kill in Vietnam solved as Taiwan ste...,Philippines president Rodrigo Duterte urges pe...,Spain arrests three Pakistanis accused of prom...,"Venezuela, where anger over food shortages is ...",A Hindu temple worker has been killed by three...,Ozone layer hole seems to be healing - US &amp...


In [5]:
data.shape, 1989 / 365

((1989, 27), 5.449315068493151)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989 entries, 0 to 1988
Data columns (total 27 columns):
Date     1989 non-null object
Label    1989 non-null int64
Top1     1989 non-null object
Top2     1989 non-null object
Top3     1989 non-null object
Top4     1989 non-null object
Top5     1989 non-null object
Top6     1989 non-null object
Top7     1989 non-null object
Top8     1989 non-null object
Top9     1989 non-null object
Top10    1989 non-null object
Top11    1989 non-null object
Top12    1989 non-null object
Top13    1989 non-null object
Top14    1989 non-null object
Top15    1989 non-null object
Top16    1989 non-null object
Top17    1989 non-null object
Top18    1989 non-null object
Top19    1989 non-null object
Top20    1989 non-null object
Top21    1989 non-null object
Top22    1989 non-null object
Top23    1988 non-null object
Top24    1986 non-null object
Top25    1986 non-null object
dtypes: int64(1), object(26)
memory usage: 419.6+ KB


### 划分训练集,测试集

In [60]:
train = data[data.Date < '2015-01-01']
test = data[data.Date > '2014-12-31']

In [61]:
X_train = train.iloc[:, 2:]
# 所有的资料库
corpus_ = X_train.values.flatten().astype(str)
X_train = X_train.values.astype(str)
# 将二维的转化为一维的,每个元素都是这一天中所有的话题的综合
X_train = np.array([' '.join(x) for x in X_train])
y_train = train.Label.values

X_test = test.iloc[:, 2:]
X_test = X_test.values.astype(str)
X_test = np.array([' '.join(x) for x in X_test])
y_test = test.Label.values


### 分词 ####

In [63]:
corpus_ = [tokenize.word_tokenize(x) for x in corpus_]
X_train = [tokenize.word_tokenize(x) for x in X_train]
X_test = [tokenize.word_tokenize(x) for x in X_test]

### 预处理
- 小写化
- 删除停止词
- 删除数字与符号
- lemma

In [74]:
stop = corpus.stopwords.words('english')
wordnet_lemmatizer = stem.WordNetLemmatizer()

def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))

# 特殊符号
def isSymbol(inputString):
    # ^\w 代表非[a-zA-Z0-9]开头的
    return bool(re.match(r'[^\w]', inputString))

def check(word):
    """
    如果需要这个单词，则True
    如果应该去除，则False
    """
    word= word.lower()
    if word in stop:
        return False
    elif hasNumbers(word) or isSymbol(word):
        return False
    else:
        return True

def preprocessing(sen):
    res = []
    for word in sen:
        if check(word):
            # 这一段的用处仅仅是去除python里面byte存str时候留下的标识。。之前数据没处理好，其他case里不会有这个情况
            word = word.lower().replace("b'", '').replace('b"', '').replace('"', '').replace("'", '')
            res.append(wordnet_lemmatizer.lemmatize(word))
    return res


In [75]:
corpus1_ = [preprocessing(x) for x in corpus_]
X_train_ = [preprocessing(x) for x in X_train]
X_tes_t = [preprocessing(x) for x in X_test]

In [76]:
print(corpus_[553])
print(corpus1_[553])

print(X_train[523])

["b'North", 'Korean', 'Leader', 'Kim', 'Jong-il', 'is', 'Confirmed', 'to', 'be', 'Ill', "'"]
['north', 'korean', 'leader', 'kim', 'jong-il', 'confirmed', 'ill']
['Two', 'redditors', 'are', 'climbing', 'Mt', '.', 'Kilimanjaro', 'for', 'charity', 'and', 'will', 'do', 'your', 'bidding', 'at', 'the', 'peak', '.', 'Do', "n't", 'squander', 'this', 'opportunity', 'to', 'let', 'your', 'upvotes', 'do', 'something', 'awesome', '.', 'It', 'is', 'estimated', 'it', 'will', 'take', '3,000', 'years', 'to', 'clear', 'Laos', 'of', 'all', 'the', 'explosive', 'remnants', 'left', 'behind', 'from', 'United', 'States', 'bombers', 'over', '30', 'years', 'ago', '.', 'More', 'than', '20,000', 'people', 'have', 'died', 'from', 'unexploded', 'ordnance', 'since', 'conflict', 'ended', '.', 'Fidel', 'to', 'Ahmadinejad', ':', "'Stop", 'Slandering', 'the', 'Jews', "'", 'Mossad', 'in', 'America', '|', 'Israels', 'intelligence', 'agencies', 'target', 'the', 'United', 'States', 'intensively', '.', 'Among', 'nations', 'c

### 训练NLP模型

In [78]:
model = word2vec.Word2Vec(corpus1_, size=128, window=5, min_count=5, workers=5)


In [80]:
# 先拿到全部的vocabulary
vocab = model.vocabulary

# 得到任意text的vector
def get_vector(word_list):
    # 建立一个全是0的array
    res =np.zeros([128])
    count = 0
    for word in word_list:
        if word in vocab:
            res += model[word]
            count += 1
    return res/count  